In [1]:
from __future__ import print_function

import os.path
import io
import os

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from apiclient.http import MediaFileUpload, MediaIoBaseDownload
from IPython.display import clear_output
import matplotlib.pyplot as plt
from PIL import Image
from ipywidgets import Button, HBox
import asyncio
import numpy as np
import glob
import IPython
import shutil
import cv2
from pigeon import annotate
from IPython.display import display

In [2]:
test = True

In [3]:
# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive']

if test:
    Unlabeled_folder_id = "'1ak6TCbKP1eIyo0PZJWcL7_bFEVpnqNzb'"
    Recycable_refund_folder_id = "'1JLMZ05T7XNk2YYMW8Lf1mJcMj3Y1K5Z5'"
    Recycable_blue_folder_id = "'1_G5GTbsntYAr-SbxLMPvzyg8Gw_Z9tqu'"
    Landfill_black_folder_id = "'1wvlEEUiiz63wHbyC9tab9NL8qEzoTrA9'"
    Compostable_green_folder_id =  "'1cxp3T7SFEOn_jHyiQQ0cxfvxLfC-B6Sw'"
else:
    Unlabeled_folder_id = "'1ak6TCbKP1eIyo0PZJWcL7_bFEVpnqNzb'"
    Recycable_refund_folder_id = "'17xsufNVLeSvrga6c5RHRqL9_y40ofoy1'"
    Recycable_blue_folder_id = "'1fYfpTL6cA81k1qdG1pni0mvleFD2rz5o'"
    Landfill_black_folder_id = "'1fu7wsN1ExBGCkte-Rt_bUIC6_8e5P-Vu'"
    Compostable_green_folder_id =  "'1DCAzFbUWhA7CnmF9HFV9RjEDkUgFqueD'"
 
folder_names = ["black", "green", "blue", "refund"]

folder_name_id = {
    Landfill_black_folder_id: folder_names[0],
    Compostable_green_folder_id: folder_names[1],
    Recycable_blue_folder_id: folder_names[2],
    Recycable_refund_folder_id: folder_names[3],
}

In [4]:
def get_current_pics_in_folder(folder):
    return [fn for fn in os.listdir(folder) if fn.endswith(('.jpeg', '.jpg'))]

In [5]:
    def get_driver_service_obj():
        """Shows basic usage of the Drive v3 API.
        Prints the names and ids of the first 10 files the user has access to.
        """
        creds = None
        # The file token.json stores the user's access and refresh tokens, and is
        # created automatically when the authorization flow completes for the first
        # time.
        if os.path.exists('token.json'):
            creds = Credentials.from_authorized_user_file('token.json', SCOPES)
        # If there are no (valid) credentials available, let the user log in.
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:

                flow = InstalledAppFlow.from_client_secrets_file(
                    'credentials.json', SCOPES)
                creds = flow.run_local_server(port=0)
            # Save the credentials for the next run
            with open('token.json', 'w') as token:
                token.write(creds.to_json())
                
        return build('drive', 'v3', credentials=creds)

In [6]:
# To Download Files
def downloadfiles(service, items, folder):
    
    if not os.path.isdir(folder):
        os.mkdir(folder)
        
    already_downloaded =  get_current_pics_in_folder('./Unlabeled')
    
    actually_downloaded = []
    for item in items:
        if item['name'] not in already_downloaded:
            name = item['name']
            actually_downloaded.append(name)
            print("Downloading file: " + name)
            dowid = item['id']
            request = service.files().get_media(fileId=dowid)
            fh = io.BytesIO()
            downloader = MediaIoBaseDownload(fh, request)
            done = False
            while done is False:
                status, done = downloader.next_chunk()
                print("Download %d%%." % int(status.progress() * 100))
                
            with io.open(folder + os.sep + name, 'wb') as f:
                fh.seek(0)
                f.write(fh.read())
        
    return actually_downloaded, already_downloaded


In [7]:
def list_and_download_files(service, local_folder_name, remote_folder_id):            
    try:
        items_metadata = list_files_folder(service, remote_folder_id)
        if items_metadata:
            actually_downloaded, already_downloaded = downloadfiles(service, items_metadata, local_folder_name)
        else:
            print('No files found.') 
            
    except HttpError as error:
        print(f'An error occurred: {error}')    
    
    print("Pictures actually downloaded: " + str(actually_downloaded))
    print("Pictures already in downloaded: " + str(already_downloaded))
    print("\n")
    print("Downloaded all pictures!")
    return items_metadata


In [8]:
def list_files_folder(service, folder_id):
    
     results = service.files().list(
        pageSize=1000, q=folder_id+" in parents",\
        fields="nextPageToken, files(id, name, mimeType, trashed)").execute()
        
     items_metadata = results.get('files', [])

     return items_metadata 

In [9]:
def delete_files(itens_in_remote_folder, files):
      
    #List all pics that are still not labeled
    current_pics_in_local_folder =  get_current_pics_in_folder('./Unlabeled')
    
    names_pics_in_remote_folder = []
    for item in itens_in_remote_folder:
        names_pics_in_remote_folder.append(item['name'])
        
    delete_from_remote_folder = set(names_pics_in_remote_folder) - set(current_pics_in_local_folder)
    
    if len(delete_from_remote_folder) == 0:
        print("No items to delete from remote folder")
        
    else:
        for picture in delete_from_remote_folder:
            for item in itens_in_remote_folder:
                if picture == item['name']:
                    print("deleting file with name: " + item['name'])
                    try:
                        service.files().delete(fileId=item['id']).execute()
                    except HttpError as error:
                        print('An error occurred: %s' % error)


In [10]:
def upload_files_to_folder(service, folder_id):

    folder_name = folder_name_id[folder_id] 
    local_pics = get_current_pics_in_folder(folder_name)

    items_metadata = list_files_folder(service, folder_id)
    
    print("items_metadata: " + str(items_metadata))
    
    pics_in_remote_folder=[]
    if items_metadata:
        for item in items_metadata:
            if item['trashed'] == False:
                pics_in_remote_folder.append(item['name'])
    else:
        pics_to_upload = local_pics
    
    pics_to_upload = set(local_pics) - set(pics_in_remote_folder)
    
    if len(pics_to_upload) == 0:
        print("No files to upload")
        return
    
    print("pics to upload: " + str(pics_to_upload))
    print("Uploading to remote: " + folder_id + " from local folder: " + folder_name )
    
    folder_id = folder_id[1:-1]

    for pic in pics_to_upload:    
        file_metadata = {'name':pic, 'parents':[folder_id]}
        print(file_metadata)
        media = MediaFileUpload(folder_name + os.sep + pic, mimetype='image/jpeg')
        try:        
            file = service.files().create(body=file_metadata,
                                        media_body=media,
                                        fields='id').execute()
            
            print ('Uploaded file with ID: ' + file.get('id'))
            
        except HttpError as error:
            print('An error occurred: ' + str(error))
    

In [11]:
#This generates the service object that we'll use for interacting with the Drive
service = get_driver_service_obj()

# Download all files that are not downloaded into the Unlabeled folder
items_metadata = list_and_download_files(service, "Unlabeled", Unlabeled_folder_id)

RefreshError: ('invalid_grant: Token has been expired or revoked.', {'error': 'invalid_grant', 'error_description': 'Token has been expired or revoked.'})

In [ ]:
_width = 500
_height = 500
tmp_image_name = "tmp.jpeg"
def resize_and_display(pic):
    img = cv2.imread(pic)
    resized = cv2.resize(img, dsize=(_width, _height), interpolation=cv2.INTER_CUBIC)
    cv2.imwrite(tmp_image_name, resized)
    display(IPython.display.IFrame(tmp_image_name, width=_height,height=_width))
    
# Logic to classify files
folder_name = "./Unlabeled"
current_pics_in_local_folder =  get_current_pics_in_folder(folder_name)

pic_with_folder=[]
for pic in current_pics_in_local_folder:
    pic_with_folder.append(folder_name + os.sep + pic)
       
annotations = annotate(
  pic_with_folder,
  options=folder_names,
  display_fn=lambda filename: resize_and_display(filename))

In [ ]:
os.remove(tmp_image_name)
# Move the pictures to the right folder
for note in annotations:
    shutil.move(note[0], note[1])

In [ ]:
#This will delete pictures that were labeled from the "Unlabeled" folder in the Drive
delete_files(items_metadata, service)

In [ ]:
upload_files_to_folder(service, Recycable_refund_folder_id)
upload_files_to_folder(service, Recycable_blue_folder_id)
upload_files_to_folder(service, Landfill_black_folder_id)
upload_files_to_folder(service, Compostable_green_folder_id)